# Answer Coalescence using only Automat

Strider finds and ranks all paths that fulfill the query.  Some of these answers can be very similar.  For instance, two answers may differ only by a gene node, but perhaps those genes are from the same family.  This permits a higher-level, more parsimonious description of an answer set.

In [6]:
import requests
import json


In [12]:
from datetime import datetime as dt
def query_automat(question,plate):
    url = f'https://automat.renci.org/{plate}/reasonerapi'
    t0 = dt.now()
    r = requests.post(url,json=question)
    t1= dt.now()
    print(f'It took {t1-t0} s to return')
    if r.status_code == 200:
        rj = r.json()
        #Coming from the KP we don't have a score, but AC expects it, so we're going to dummy it.
        for result in rj['results']:
            result['score'] = 1
        return rj
    print(r.status_code)
    return None

In [13]:
def create_question(disease_curie, relation_type):
    question = { "nodes": [{"id":"n0", "type":"disease", "curie":disease_curie},
                          {"id": "n1", "type":"chemical_substance"}],
                 "edges": [ {"id": "e0", "source_id": "n1", "target_id": "n0", "type": relation_type}]}
    return {'query_graph':question}


In [36]:
def getlabel(curie):
    url = 'https://nodenormalization-sri.renci.org/get_normalized_nodes'
    r = requests.get(url,params={'curie':curie}).json()
    try:
        return r[curie]['id']['label']
    except:
        return ''

## Property Coalescence

Find drugs that treat diabetes in mychem.

In [14]:
#disease = 'MONDO:0005180'  #Parkinson
#disease = 'MONDO:0004979'  #Asthma
disease = "MONDO:0005148"   #T2D
edgetype="treats"
question = create_question(disease,edgetype)
res = query_automat(question,'mychem')

It took 0:00:38.578359 s to return


In [15]:
url = 'https://answercoalesce.renci.org/coalesce'
params = {'method':'property'}
r = requests.post(url,json=res,params=params)
print(r.status_code)

dict_keys(['query_graph', 'results', 'knowledge_graph'])
200


In [16]:
results = r.json()['results']

In [24]:
print(results[0]['node_bindings'][1])

{'qg_id': 'n1', 'kg_id': ['CHEBI:27999', 'CHEBI:71272', 'CHEBI:6801', 'CHEBI:6935', 'CHEBI:31660', 'CHEBI:5383', 'CHEBI:136042', 'CHEBI:135545', 'CHEBI:134731', 'CHEBI:68610', 'CHEBI:136052', 'CHEBI:5384', 'CHEBI:40237', 'CHEBI:32300', 'CHEBI:82720', 'CHEBI:134724', 'CHEBI:135349', 'CHEBI:135285', 'CHEBI:72323', 'CHEBI:9613', 'CHEBI:5441', 'CHEBI:3209', 'CHEBI:3650', 'CHEBI:134735', 'CHEBI:136043', 'CHEBI:93416', 'CHEBI:85078', 'CHEBI:73274', 'CHEBI:9753', 'CHEBI:8805', 'CHEBI:31897', 'CHEBI:8228', 'CHEBI:31654', 'CHEBI:28052', 'CHEBI:135731'], 'coalescence_method': 'property_enrichment', 'p_values': [1.196277654223006e-130], 'properties': ['Blood Glucose Lowering Agents']}


In [26]:
for result in results:
    chemnode = result['node_bindings'][1]
    print(len(chemnode['kg_id']), chemnode['p_values'], chemnode['properties'])

35 [1.196277654223006e-130] ['Blood Glucose Lowering Agents']
30 [4.936115949370616e-120] ['Oral Hypoglycemics']
31 [7.425481227210775e-120, 1.7111072995800748e-87] ['Drugs Used in Diabetes', 'Alimentary Tract and Metabolism']
22 [1.7797817994985986e-67] ['hypoglycemic_agent']
25 [4.5954180236113555e-45] ['drugbank.investigational']
11 [7.84512571106143e-45] ['Sulfonylureas']
26 [3.6061819505761044e-44] ['therapeutic_flag']
13 [7.432631862414306e-41] ['Hypoglycemia-Associated Agents']
18 [6.9662039084864395e-40] ['Sulfur Compounds']
23 [7.279401756016781e-40] ['drugbank.approved']
21 [7.356562576276e-40] ['oral']
15 [2.6355189664578375e-38] ['Cytochrome P-450 CYP2C9 Substrates']
9 [7.541689787649391e-34, 4.659672044972656e-27] ['DPP-IV Inhibitors', 'Agents causing angioedema']
12 [5.561853262966284e-32] ['Sulfones']
10 [3.7557413407340904e-31] ['Urea']
23 [3.889371075810866e-31, 5.329692091072344e-31, 3.3504554405901985e-27] ['drug', 'pharmaceutical', 'application']
7 [1.25927542021571

## Ontology Coalescence

In [27]:
url = 'https://answercoalesce.renci.org/coalesce'
params = {'method':'ontology'}
r = requests.post(url,json=res,params=params)
print(r.status_code)

200


In [28]:
results = r.json()['results']
print(len(results))

3


In [29]:
print(results[0]['node_bindings'][1])

{'qg_id': 'n1', 'kg_id': ['CHEBI:5384', 'CHEBI:28052', 'CHEBI:9613', 'CHEBI:5441', 'CHEBI:3650', 'CHEBI:135118', 'CHEBI:5383', 'CHEBI:31654', 'CHEBI:135731', 'CHEBI:27999'], 'coalescence_method': 'ontology_enrichment', 'p_value': 2.7218132323383963e-08, 'superclass': 'CHEBI:35358'}


In [39]:
for result in results:
    chemnode = result['node_bindings'][1]
    sclass = chemnode['superclass']
    label = getlabel(sclass)
    print(len(chemnode['kg_id']), 'superclass:',sclass,label)

10 superclass: CHEBI:35358 sulfonamide
13 superclass: CHEBI:33261 organosulfur compound
8 superclass: CHEBI:76983 N-sulfonylurea


## Graph Coalescence

In [40]:
url = 'https://answercoalesce.renci.org/coalesce'
params = {'method':'graph'}
r = requests.post(url,json=res,params=params)
print(r.status_code)

200


In [41]:
r.json()

{'query_graph': {'nodes': [{'id': 'n0',
    'type': 'disease',
    'curie': 'MONDO:0005148'},
   {'id': 'n1', 'type': 'chemical_substance'}],
  'edges': [{'id': 'e0',
    'source_id': 'n1',
    'target_id': 'n0',
    'type': 'treats'}]},
 'knowledge_graph': {'nodes': [{'potassium_channel_blocker': True,
    'iupac_name': '1-(azepan-1-yl)-3-tosyl-urea',
    'pharmaceutical': True,
    'role': True,
    'inchikey': 'OUDSBRTVNLOZBN-UHFFFAOYSA-N',
    'smiles': 'Cc1ccc(S(=O)(=O)NC(=O)NN2CCCCCC2)cc1',
    'drugbank.accession_number': 'APRD01267',
    'mass': '311.40000',
    'molecular_weight': 311.4,
    'potassium_channel_modulator': True,
    'drug': True,
    'hypoglycemic_agent': True,
    'simple_smiles': 'Cc1ccc(S(=O)(=O)NC(=O)NN2CCCCCC2)cc1',
    'chebi.orig_smiles': 'Cc1ccc(cc1)S(=O)(=O)NC(=O)NN1CCCCCC1',
    'id': 'CHEBI:9613',
    'membrane_transport_modulator': True,
    'molecular_formula': 'C14H21N3O3S',
    'pubchem.orig_smiles': 'CC1=CC=C(C=C1)S(=O)(=O)NC(=O)NN2CCCCCC2',
   

In [44]:
def create_proc_question(disease_curie, relation_type):
    question = { "nodes": [{"id":"n0", "type":"disease", "curie":disease_curie},
                          {"id": "n1", "type":"biological_process_or_activity"}],
                 "edges": [ {"id": "e0", "source_id": "n0", "target_id": "n1", "type": relation_type}]}
    return {'query_graph':question}

disease = "MONDO:0005148"   #T2D
edgetype="disrupts"
question = create_proc_question(disease,edgetype)
res = query_automat(question,'uberon')

It took 0:00:00.216365 s to return


In [45]:
res

{'query_graph': {'nodes': [{'id': 'n0',
    'type': 'disease',
    'curie': 'MONDO:0005148'},
   {'id': 'n1', 'type': 'biological_process_or_activity'}],
  'edges': [{'id': 'e0',
    'source_id': 'n0',
    'target_id': 'n1',
    'type': 'disrupts'}]},
 'results': [],
 'knowledge_graph': {'nodes': [], 'edges': []}}